<a href="https://colab.research.google.com/github/sagacemente/PAN_2023_crypto/blob/main/PAN_CRYPTO_Run_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import an install neded modules

In [1]:
!pip install simpletransformers
!pip install tensorflow-addons
!pip install -U deep-translator
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 26

In [2]:
import numpy as np
import torch
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
import tensorflow_addons as tfa
import os
import pandas as pd
import json
import sklearn
from sklearn.preprocessing import OneHotEncoder
from urllib import request
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold, StratifiedKFold

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
from deep_translator import GoogleTranslator
import tensorflow as tf
from urllib import request
import os
import json
import pandas as pd
import shutil

# Import class Vectorizer
module_url = f"https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_vectorzer.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_vectorzer import Vectorizer

# Import class Dataset
module_url = f"https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_dataset.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_dataset import Dataset

#Import class Simulator
module_url = f"https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/simulator_kfold.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from simulator_kfold import Simulator

Fetching https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_vectorzer.py
Fetching https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_dataset.py
Fetching https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/simulator_kfold.py
Fetching https://raw.githubusercontent.com/sagacemente/PAN_2023_crypto/main/crypto_vectorzer.py


# Load Dataset 

In [4]:
url = 'https://github.com/sagacemente/PAN_2023_crypto/raw/main/pan23-profiling-cryptocurrency-influencers-augmented-it.zip'
ds = Dataset(url, n_subtask = 1, language='it')
ds.build_ds(1)
xtr, xte = ds.get_train_test_df()

# CHECK SMAPLES
N = 2
for i in ds.test_set.take(N):
  print(i)

224710/224710 [==============================] - 0s 0us/step
id_label_dict {'0037a672f0ed64b3231bac64853a278d': 'nano', '03eaa72711143b521c073d9ac5745923': 'nano', '0409fe210a0edfe258d21e3404e1ce05': 'micro', '05ca545f2f700d0d5c916657251d010b': 'mega', '062492818c984febba843b650a4a602e': 'nano', '0d3700fa5c7c3fce6fd1e1ffd5282f50': 'macro', '0e1d2c43b93e8e80dc8eb6b29d48b2c1': 'no influencer', '0ed9637249db91cb2c256ec156ce1977': 'no influencer', '0f0942696ae8bcadf0db494cce7333e0': 'nano', '11727505346c5f966891f74094042073': 'nano', '1316cba6e1442a126102d5e2ab0813ec': 'macro', '13e873428de3ccfd3873b3d7258ad411': 'macro', '179aa6d5d94999ae0bf77d13bcc5fa31': 'mega', '1bc1201be24ca9d0c0afd94ca0116609': 'nano', '1bc3271b139a2c06af59e579890796f7': 'nano', '1cbf16d93cc6a672eaed9c16ac07f114': 'mega', '1fe087134fe681f8509f8d9ced432621': 'macro', '21658f2f073f0e041b79bf0f7632a0eb': 'mega', '238a39478f2f7b782dfdbeee63df503f': 'no influencer', '28ab93d1cfa1452bee4da4a93746b956': 'micro', '2a45947271

# Run Electra

In [ ]:
simulator = Simulator("xlnet", num_fold =5, nr_epochs=10, ds=ds, vectorize_layer=['NO'], num_labels=5)
simulator.run()


Setup for Roberta completed.
FOLD NUM 1


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a


EPOCH NUMBER:  0

NOW TRAIN THE MODEL.


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/128 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/128 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/8 [00:00<?, ?it/s]